# 1. Data Collection - PART 2
## Notebook to scrap each page 
We generate links based on the stockno that we grabed and stored into car_list.csv file. to get further details we click the 'see all specifications' button and scrap some more 

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import math
import os
chromedriver = f"{os.environ['HOME']}/.local/bin/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

### 1. Opening our car_list.csv

In [2]:
prod_list=pd.read_csv('../data/scrape/product_links_tv_stands.csv')
prod_list.drop('Unnamed: 0',axis =1, inplace = True)
print(prod_list.shape)
prod_list.tail()                    

(1912, 2)


,link,type
1907,https://www.overstock.com/Home-Garden/Somette-...,tv stands
1908,https://ad.tagdelivery.com/click?id=a8ba0d79-c...,tv stands
1909,https://ad.tagdelivery.com/click?id=d6216692-4...,tv stands
1910,https://ad.tagdelivery.com/click?id=2844e3cf-e...,tv stands
1911,https://ad.tagdelivery.com/click?id=872fd552-8...,tv stands


### 2. Setting up all the headings/parameters I need and an empty dataframe 

In [15]:
# list out all heading, parameters we will get from scraping a page. 
# add stock_no so that later on i can join the car_list.csv and car_details.csv  
heading1 =['page_link','product_name','item_no','description','style',\
          'material','type_class','color','rating']

#initialize an empty dataframe
df_product=pd.DataFrame(columns=heading1)
df_product

,page_link,product_name,item_no,description,style,material,type_class,color,rating


In [16]:
# list out all heading, parameters we will get from scraping a page. 
# add stock_no so that later on i can join the car_list.csv and car_details.csv  
heading2 =['image_link','product_name','page_link']

#initialize an empty dataframe
df_images=pd.DataFrame(columns=heading2)
df_images

,image_link,product_name,page_link


### 2. Scrap it

In [17]:
#counter to check where we are, track if we ge interrupted
counter=1001

In [18]:
# condition for x_path
#row_selector = '//div[@class ="card--text-block-list--row"]'

driver = webdriver.Chrome(chromedriver)
#loop through the car_list we have made.  maually set start and finish. Use exit counter to know where you are at 
for link_str in prod_list.loc[1001:,'link']:
    
    print('----START PAGE ----')
    print(link_str)
    print('COUNT= ',counter)
    driver.get(str(link_str))
    time.sleep(2)
        
    page_link = link_str
    
    # get product name
    product_name_selector = '//h1'
    product_name = driver.find_element_by_xpath(product_name_selector).text
    #print(product_name)
    
    # get item_no
    item_no_selector ='//p[@class ="description-item-number"]'
    item_no=driver.find_element_by_xpath(item_no_selector).text.split()[1]
    item_no = int(item_no)
    #print(item_no)
    
    # get description
    description_selector ='//span[@itemprop ="description"]'
    description=driver.find_element_by_xpath(description_selector).text.split('\n')[0]
    #print(description)
    
    # get style , material, type_class, color
    style_selector = '//td[contains(text(),"Style")]/following-sibling::td'
    material_selector = '//td[contains(text(),"Material")]/following-sibling::td'
    type_class_selector = '//td[contains(text(),"Type")]/following-sibling::td'
    color_selector = '//td[contains(text(),"Color")]/following-sibling::td'
    
    try:
        style = driver.find_element_by_xpath(style_selector).text.strip()
    except:
        style = ""       
    try:
        material = driver.find_element_by_xpath(material_selector).text.strip()
    except:
        material = ""    
    try:
        type_class = driver.find_element_by_xpath(type_class_selector).text.strip()
    except:
        type_class = ""
    try:
        color = driver.find_element_by_xpath(color_selector).text.strip()
    except:
        color = ""

    #print(style,material,color,type_class)
    
    # get rating
    rating_selector ='//span[@class ="overall-rating"]'
    try:
        rating=driver.find_element_by_xpath(rating_selector).text
    except:
        rating = 0
    
    rating = float(rating)
    #print(rating)

    temp_list = [page_link,product_name,item_no,description,style,material,type_class,color,rating]
    try:
        # put into a  first panda
        df_product = df_product.append(pd.DataFrame([temp_list], columns=heading1),ignore_index=True)
    except:
        print('OPPS A ... no put in pandas')
    

    ###### images ##
    image_panel = '//div[@class="gallery-controls"]//div[@class ="thumb-frame"]//child::li'
    for row in driver.find_elements_by_xpath(image_panel):
        image_link = row.get_attribute('data-max-img')
        temp_img_list = [image_link,product_name,page_link]
        try:
        # put into a second panda
            df_images = df_images.append(pd.DataFrame([temp_img_list], columns=heading2),ignore_index=True)
        except:
            print('OPPS B ... no put in pandas')      
    
    #print(len(driver.find_elements_by_xpath(image_panel)))
    counter +=1
    print('----END PAGE-----')
    
driver.close()
  

----START PAGE ----
https://www.overstock.com/Home-Garden/Convenience-Concepts-Designs2Go-Key-Largo-TV-Stand/12014079/product.html?refccid=ZGZXUYFP5ARJHHSEJCIHUQ2QTI&searchidx=949
COUNT=  1001
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/CorLiving-Bakersfield-Ravenwood-Black-TV-Component-Stand/8969134/product.html?refccid=ZGZXUYFP5ARJHHSEJCIHUQ2QTI&searchidx=950
COUNT=  1002
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Oliver-James-Clair-Oak-TV-Stand/20133325/product.html?refccid=ZGZXUYFP5ARJHHSEJCIHUQ2QTI&searchidx=951
COUNT=  1003
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/K-B-Contemporary-Black-Entertainment-Center/14741543/product.html?refccid=ZGZXUYFP5ARJHHSEJCIHUQ2QTI&searchidx=952
COUNT=  1004
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Atlantic-Nantucket-Espresso-50-inch-2-drawer-Adjustable-shelf-Entertainment-Console/14012401/product.html?refccid=ZGZXU

----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Gable-Black-72-inch-RTA-Entertainment-TV-Stand/19886010/product.html?refccid=CNHP2M6DQPYYWR2GN6WUBJLPCE&searchidx=977
COUNT=  1033
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Somette-Millstone-Textured-Ivory-Three-Door-Media-Credenza/22164054/product.html?refccid=CNHP2M6DQPYYWR2GN6WUBJLPCE&searchidx=978
COUNT=  1034
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Somette-Mansfield-Black-and-Silver-Four-Door-Media-Credenza/22164665/product.html?refccid=CNHP2M6DQPYYWR2GN6WUBJLPCE&searchidx=979
COUNT=  1035
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/PINETO-Collection-High-Gloss-White-Lacquer-Entertainment-Center-by-Casabianca-Home/10514179/product.html?refccid=CNHP2M6DQPYYWR2GN6WUBJLPCE&searchidx=980
COUNT=  1036
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Somette-Millstone-Textured-Ivory-Me

----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Avenue-Greene-Naperville-TV-Stand-for-TVs-up-to-65-inch/23623312/product.html?refccid=CNHP2M6DQPYYWR2GN6WUBJLPCE&searchidx=1016
COUNT=  1072
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Accentuations-by-Manhattan-Comfort-Salem-Mid-Century-Style-TV-Stand/12710870/product.html?refccid=CNHP2M6DQPYYWR2GN6WUBJLPCE&searchidx=1017
COUNT=  1073
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Contemporary-Brown-TV-Stand-by-Baxton-Studio/9958414/product.html?refccid=CNHP2M6DQPYYWR2GN6WUBJLPCE&searchidx=1018
COUNT=  1074
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Gwendolyn-Modern-Faux-Wood-TV-Stand-by-Christopher-Knight-Home/20746982/product.html?refccid=CNHP2M6DQPYYWR2GN6WUBJLPCE&searchidx=1019
COUNT=  1075
----END PAGE-----
----START PAGE ----
https://ad.tagdelivery.com/click?id=a8ba0d79-c2c5-4119-b9c6-2a9f1479697d&x=GhkMbkSjbVir

----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/South-Mountain-Farmhouse-Plasma-Media-Console/18155616/product.html?refccid=33GRCME3Z5MJBBCNYVAVKSOID4&searchidx=1044
COUNT=  1104
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Gable-Red-72-inch-RTA-Entertainment-TV-Stand/19886031/product.html?refccid=33GRCME3Z5MJBBCNYVAVKSOID4&searchidx=1045
COUNT=  1105
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Miranda-Media-Stand/15033359/product.html?refccid=33GRCME3Z5MJBBCNYVAVKSOID4&searchidx=1046
COUNT=  1106
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Isola-2-section-Modern-TV-Wall-Unit/25410500/product.html?refccid=33GRCME3Z5MJBBCNYVAVKSOID4&searchidx=1047
COUNT=  1107
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Furniture-of-America-Thorne-Antique-Oak-Industrial-60-inch-TV-Stand/9914783/product.html?refccid=33GRCME3Z5MJBBCNYVAVKSOID4&searchidx

----END PAGE-----
----START PAGE ----
https://ad.tagdelivery.com/click?id=2844e3cf-e480-45ae-943a-938a3553e766&x=BOnWv4-CxCnZhQCmBEH9VP5I_imKYVNmZkG1vkNfa_WWpPCnwZnbxzRcbBYCnreubnTduOci23_bKpytcSwOo2WqWkmmq2QUjPKKNWDN6d7emXdA_Wpolayh-xRJ5Om0Mpv9rBu86FPzmgriMPDYqRt_5tF4bXFgJ6FtvOuiAsD2noIvmTcnRcXnQ6whA57WToTMcBQM-rtrBZ_xtVuyksdq2cnf7WG6LEkgy-_cNstMRIoUn9XGsMRL8h4LvLxP2Zn38xZH7dqagn7RCgYFvg%3D%3D&redirect=https%3A%2F%2Fwww.overstock.com%2FHome-Garden%2FTopanga-Reclaimed-Wood-4-Drawer-TV-Stand%2F16257270%2Fproduct.html%3Frefccid%3DTXFTEHGJENS22SKD2TDSQQ4ULM%26searchidx%3D2
COUNT=  1142
----END PAGE-----
----START PAGE ----
https://ad.tagdelivery.com/click?id=872fd552-8efe-4a45-9152-756fb7820c9e&x=gxTCn69ZeqhchqiJbbYGvC1lpN_cz7m0DJ2d7ReNs3f9K8N8psG49FPmcqSd6p2bppBYRcxt3TtzTQUigCR1I36zbeE3aA3yQC5vyfhFKr0ZnCVnCnIDKjSVukOt1NTn3kipXrTvSEI32Zf_dx2BbGyfPZMOrjkbVO0oXrSG8LGft-p7gCsGV69TTRszAtL0z4qg4x_7z8IG9UOMht0wTNMFSnU4or1kk45lcqCwFlVwDc0vqsCcVz75IRyJ2jCmOrks6_i3RZg_BB0Tlp2sZA%3D%3D&redirect=htt

...

----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Ventura-Black-Cherry-60-inch-Entertainment-TV-Console/19980954/product.html?refccid=D6VAIXOUVHXKK32NAVVET2RH2A&searchidx=1646
COUNT=  1746
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Giverny-African-Honey-52-inch-Entertainment-TV-Console-52-inches/19886948/product.html?refccid=D6VAIXOUVHXKK32NAVVET2RH2A&searchidx=1647
COUNT=  1747
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Consoles-Country-Pine-Entertainment-Center-Kit-31H-Renovators-Supply/13300768/product.html?refccid=D6VAIXOUVHXKK32NAVVET2RH2A&searchidx=1648
COUNT=  1748
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Striking-Spacious-Modern-TV-Stand-Buffet-Black/18088704/product.html?refccid=D6VAIXOUVHXKK32NAVVET2RH2A&searchidx=1649
COUNT=  1749
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/East-Shore-Black-Cherry-Contemporary-Entertai

----END PAGE-----
----START PAGE ----
https://ad.tagdelivery.com/click?id=2844e3cf-e480-45ae-943a-938a3553e766&x=BOnWv4-CxCnZhQCmBEH9VP5I_imKYVNmZkG1vkNfa_WWpPCnwZnbxzRcbBYCnreubnTduOci23_bKpytcSwOo2WqWkmmq2QUjPKKNWDN6d7emXdA_Wpolayh-xRJ5Om0Mpv9rBu86FPzmgriMPDYqRt_5tF4bXFgJ6FtvOuiAsD2noIvmTcnRcXnQ6whA57WToTMcBQM-rtrBZ_xtVuyksdq2cnf7WG6LEkgy-_cNstMRIoUn9XGsMRL8h4LvLxP2Zn38xZH7dqagn7RCgYFvg%3D%3D&redirect=https%3A%2F%2Fwww.overstock.com%2FHome-Garden%2FTopanga-Reclaimed-Wood-4-Drawer-TV-Stand%2F16257270%2Fproduct.html%3Frefccid%3DTXFTEHGJENS22SKD2TDSQQ4ULM%26searchidx%3D2
COUNT=  1782
----END PAGE-----
----START PAGE ----
https://ad.tagdelivery.com/click?id=872fd552-8efe-4a45-9152-756fb7820c9e&x=gxTCn69ZeqhchqiJbbYGvC1lpN_cz7m0DJ2d7ReNs3f9K8N8psG49FPmcqSd6p2bppBYRcxt3TtzTQUigCR1I36zbeE3aA3yQC5vyfhFKr0ZnCVnCnIDKjSVukOt1NTn3kipXrTvSEI32Zf_dx2BbGyfPZMOrjkbVO0oXrSG8LGft-p7gCsGV69TTRszAtL0z4qg4x_7z8IG9UOMht0wTNMFSnU4or1kk45lcqCwFlVwDc0vqsCcVz75IRyJ2jCmOrks6_i3RZg_BB0Tlp2sZA%3D%3D&redirect=htt

----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Signature-Design-by-Ashley-Ralene-Medium-Brown-TV-Stand/13467029/product.html?refccid=JENXSJS57WNKRV2CZ5C32ZUCDI&searchidx=1714
COUNT=  1818
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Monarch-White-Open-Concept-TV-Console/10746655/product.html?refccid=JENXSJS57WNKRV2CZ5C32ZUCDI&searchidx=1715
COUNT=  1819
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/White-TV-Console/8280675/product.html?refccid=JENXSJS57WNKRV2CZ5C32ZUCDI&searchidx=1716
COUNT=  1820
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Glass-MDF-and-Metal-48-inch-Long-Warm-Cherry-TV-Stand/12850417/product.html?refccid=JENXSJS57WNKRV2CZ5C32ZUCDI&searchidx=1717
COUNT=  1821
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Contemporary-TV-Console-with-Open-Storage-Brown/18853207/product.html?refccid=JENXSJS57WNKRV2CZ5C32ZUCDI&searchidx=1

----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Astonishing-Entertainment-TV-Stand-Black/19391072/product.html?refccid=M6FBP2TQXKC3NU2MU34F6R3BMM&searchidx=1743
COUNT=  1851
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Attractive-Transitional-Style-TV-Console-Brown/18853208/product.html?refccid=M6FBP2TQXKC3NU2MU34F6R3BMM&searchidx=1744
COUNT=  1852
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Stunning-black-Tv-console-With-Designer-Legs/18853210/product.html?refccid=M6FBP2TQXKC3NU2MU34F6R3BMM&searchidx=1745
COUNT=  1853
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Boahaus-Brown-Wood-65-inch-TV-Stand/19470063/product.html?refccid=M6FBP2TQXKC3NU2MU34F6R3BMM&searchidx=1746
COUNT=  1854
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Casual-Contemporary-Metal-TV-Console-with-Bracket-Black/18853249/product.html?refccid=M6FBP2TQXKC3NU2MU34F6R3BM

----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/VIGO-New-High-Gloss-TV-Stand-White-White/25417487/product.html?refccid=M6FBP2TQXKC3NU2MU34F6R3BMM&searchidx=1782
COUNT=  1890
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/VIGO-High-Gloss-TV-Stand-White-White/25417578/product.html?refccid=M6FBP2TQXKC3NU2MU34F6R3BMM&searchidx=1783
COUNT=  1891
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Everest-Collection-Filing-Storage-Credenza/25443867/product.html?refccid=M6FBP2TQXKC3NU2MU34F6R3BMM&searchidx=1784
COUNT=  1892
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Offex-60-Industrial-TV-Media-Stand-Console-with-Mesh-Rustic-Oak/23564291/product.html?refccid=M6FBP2TQXKC3NU2MU34F6R3BMM&searchidx=1785
COUNT=  1893
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Brassex-14858-Black-4-tier-Pier-TV-Stand-with-Drawers/15870884/product.html?refccid=M6FBP2TQXKC

In [19]:
# what have I got ?
print(counter)
print(df_product.shape)
df_product.tail()

1912
(911, 9)


,page_link,product_name,item_no,description,style,material,type_class,color,rating
906,https://www.overstock.com/Home-Garden/Somette-...,Somette Epic Two Tone Rub Three Door Media Cre...,27825414,The Epic Two Tone Rub finish is a stunning con...,Modern & Contemporary,"MDF, Wood","Entertainment Center, Media Cabinets, Storage",Brown,0.0
907,https://ad.tagdelivery.com/click?id=a8ba0d79-c...,Clay Alder Home Sachs Modern 2-drawer TV Stand...,20254757,Give your living room decor a modern update wi...,Transitional,"Cherry, MDF, Veneer","Media Cabinets, Storage, TV Stands","Black, Brown",4.4
908,https://ad.tagdelivery.com/click?id=d6216692-4...,Haight Media TV Stand,23120193,Haight Media TV Stand is beautifully designed ...,Rustic,"Metal, Veneer, Wood","Entertainment Center, TV Console, TV Stands",Grey,0.0
909,https://ad.tagdelivery.com/click?id=2844e3cf-e...,Topanga Reclaimed Wood 4 Drawer TV Stand,22623357,Topanga Reclaimed Wood Four Drawer TV Stand ex...,"Industrial, Rustic, Urban","Iron, Metal, Reclaimed Wood, Wood","Media Cabinets, Storage, TV Stands",Brown,4.4
910,https://ad.tagdelivery.com/click?id=872fd552-8...,"Dresden 48"" Antique Gray Media Console - 50 in...",25793330,Dresden Media TV Stand is expertly crafted fro...,Urban,Wood,"Entertainment Units, TV Console, TV Stands",Grey,4.0


In [20]:
df_product['page_link'].nunique()

855

In [21]:
print(df_images['page_link'].nunique())

855


In [22]:
df_images.tail()

,image_link,product_name,page_link
3933,https://ak1.ostkcdn.com/images/products/198647...,"Dresden 48"" Antique Gray Media Console - 50 in...",https://ad.tagdelivery.com/click?id=872fd552-8...
3934,https://ak1.ostkcdn.com/images/products/198647...,"Dresden 48"" Antique Gray Media Console - 50 in...",https://ad.tagdelivery.com/click?id=872fd552-8...
3935,https://ak1.ostkcdn.com/images/products/198647...,"Dresden 48"" Antique Gray Media Console - 50 in...",https://ad.tagdelivery.com/click?id=872fd552-8...
3936,https://ak1.ostkcdn.com/images/products/198647...,"Dresden 48"" Antique Gray Media Console - 50 in...",https://ad.tagdelivery.com/click?id=872fd552-8...
3937,https://ak1.ostkcdn.com/images/products/198647...,"Dresden 48"" Antique Gray Media Console - 50 in...",https://ad.tagdelivery.com/click?id=872fd552-8...


### 3. Write my data frame to file
If the file does not exist then i write the dataframe with headers else if the file is already there then don't append with the header row 

In [23]:
file_name1 = '../data/scrape/product_details_tv_stand.csv'
from pathlib import Path
config = Path(file_name1)
if config.is_file():
    print('exist = yes')
    with open(file_name1, 'a') as f:
        df_product.to_csv(f, header=False)
    # Store configuration file values
else:
    # Keep presets
    print('exists = no')
    with open(file_name1, 'a') as f:
        df_product.to_csv(f, header=True)
    

exist = yes


In [24]:
file_name2 = '../data/scrape/image_data_tv_stand.csv'
from pathlib import Path
config = Path(file_name2)
if config.is_file():
    print('exist = yes')
    with open(file_name2, 'a') as f:
        df_images.to_csv(f, header=False)
    # Store configuration file values
else:
    # Keep presets
    print('exists = no')
    with open(file_name2, 'a') as f:
        df_images.to_csv(f, header=True)
    

exist = yes


### 4. Check it
read the file and check if stuff was written 

In [25]:
df_check1 = pd.read_csv(file_name1)
print(df_check1.shape)
df_check1.tail()

(1912, 10)


,Unnamed: 0,page_link,product_name,item_no,description,style,material,type_class,color,rating
1907,906,https://www.overstock.com/Home-Garden/Somette-...,Somette Epic Two Tone Rub Three Door Media Cre...,27825414,The Epic Two Tone Rub finish is a stunning con...,Modern & Contemporary,"MDF, Wood","Entertainment Center, Media Cabinets, Storage",Brown,0.0
1908,907,https://ad.tagdelivery.com/click?id=a8ba0d79-c...,Clay Alder Home Sachs Modern 2-drawer TV Stand...,20254757,Give your living room decor a modern update wi...,Transitional,"Cherry, MDF, Veneer","Media Cabinets, Storage, TV Stands","Black, Brown",4.4
1909,908,https://ad.tagdelivery.com/click?id=d6216692-4...,Haight Media TV Stand,23120193,Haight Media TV Stand is beautifully designed ...,Rustic,"Metal, Veneer, Wood","Entertainment Center, TV Console, TV Stands",Grey,0.0
1910,909,https://ad.tagdelivery.com/click?id=2844e3cf-e...,Topanga Reclaimed Wood 4 Drawer TV Stand,22623357,Topanga Reclaimed Wood Four Drawer TV Stand ex...,"Industrial, Rustic, Urban","Iron, Metal, Reclaimed Wood, Wood","Media Cabinets, Storage, TV Stands",Brown,4.4
1911,910,https://ad.tagdelivery.com/click?id=872fd552-8...,"Dresden 48"" Antique Gray Media Console - 50 in...",25793330,Dresden Media TV Stand is expertly crafted fro...,Urban,Wood,"Entertainment Units, TV Console, TV Stands",Grey,4.0


In [26]:
df_check2 = pd.read_csv(file_name2)
print(df_check2.shape)
print(df_check2['page_link'].nunique())
df_check2.tail()

(10884, 4)
1804


,Unnamed: 0,image_link,product_name,page_link
10879,3933,https://ak1.ostkcdn.com/images/products/198647...,"Dresden 48"" Antique Gray Media Console - 50 in...",https://ad.tagdelivery.com/click?id=872fd552-8...
10880,3934,https://ak1.ostkcdn.com/images/products/198647...,"Dresden 48"" Antique Gray Media Console - 50 in...",https://ad.tagdelivery.com/click?id=872fd552-8...
10881,3935,https://ak1.ostkcdn.com/images/products/198647...,"Dresden 48"" Antique Gray Media Console - 50 in...",https://ad.tagdelivery.com/click?id=872fd552-8...
10882,3936,https://ak1.ostkcdn.com/images/products/198647...,"Dresden 48"" Antique Gray Media Console - 50 in...",https://ad.tagdelivery.com/click?id=872fd552-8...
10883,3937,https://ak1.ostkcdn.com/images/products/198647...,"Dresden 48"" Antique Gray Media Console - 50 in...",https://ad.tagdelivery.com/click?id=872fd552-8...


### next -> clean_data.ipynb 